## MongoDB Connection

In [ ]:
# Import required libraries for embeddings, LLM, and environment configuration
from langchain_ollama import OllamaEmbeddings
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

# Initialize the embedding model and LLM

embedding = OllamaEmbeddings(model= "nomic-embed-text:v1.5")
llm = ChatGroq(model="openai/gpt-oss-120b")

In [ ]:
# Test the embedding model by embedding a sample text
r = embedding.embed_documents("Bye bye")
len(r[0])  # Print the vector dimension

768

In [ ]:
# Load MongoDB Atlas connection URI from environment variables
# This URI contains credentials for connecting to the MongoDB Atlas cluster
import os
MONGODB_ATLAS_CLUSTER_URI = os.environ.get("MONGODB_ATLAS_CLUSTER_URI")

In [70]:
# Set up MongoDB Atlas vector store for similarity search
# This integrates LangChain with MongoDB Atlas for vector-based document retrieval
from langchain_mongodb import MongoDBAtlasVectorSearch
from pymongo import  MongoClient

# Connect to MongoDB Atlas cluster using the connection URI
client = MongoClient(MONGODB_ATLAS_CLUSTER_URI)

# Define database and collection names for storing vectors and documents
DB_NAME = "rag"
COLLECTION_NAME = "langchain_rag_vector_store"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "langchain-test-index-vectorstores"

# Reference the MongoDB collection
MONGODB_COLLECTION = client[DB_NAME][COLLECTION_NAME]

vector_store = MongoDBAtlasVectorSearch(
    collection=MONGODB_COLLECTION,
    embedding=embedding,
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
    relevance_score_fn="cosine",
)

# Create vector search index on the collection (768-dimensional vectors)
vector_store.create_vector_search_index(dimensions=768)

In [71]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

def load_documents(data_dir):
    documents = []
    for file in os.listdir(data_dir):
        file_path = os.path.join(data_dir, file)
        if file.endswith(".pdf"):
            # Load PDF using PyPDFLoader
            loader = PyPDFLoader(file_path)
            documents.extend(loader.load())
        # Note: Support for DOCX files can be added by uncommenting the section below
        # elif file.endswith(".docx"):
        #     loader = Docx2txtLoader(file_path)
        #     documents.extend(loader.load())
    return documents

In [72]:
# Load all PDF documents from the 'data' directory
docs = load_documents("data")
len(docs)

3

In [73]:
# Split documents into smaller chunks for efficient embedding and retrieval
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)
len(chunks)  # Display the total number of chunks created

15

In [ ]:
# Add all document chunks to the MongoDB vector store
vector_store.add_documents(chunks)

['69331a25e7928100e9416fa0',
 '69331a25e7928100e9416fa1',
 '69331a25e7928100e9416fa2',
 '69331a25e7928100e9416fa3',
 '69331a25e7928100e9416fa4',
 '69331a25e7928100e9416fa5',
 '69331a25e7928100e9416fa6',
 '69331a25e7928100e9416fa7',
 '69331a25e7928100e9416fa8',
 '69331a25e7928100e9416fa9',
 '69331a25e7928100e9416faa',
 '69331a25e7928100e9416fab',
 '69331a25e7928100e9416fac',
 '69331a25e7928100e9416fad',
 '69331a25e7928100e9416fae']

In [ ]:
# Perform a test similarity search to verify the vector store is working
results = vector_store.similarity_search(
    "RAG", k=4
)

In [75]:
# Display the content of the retrieved documents
results[0].page_content  # First result
results[1].page_content  # Second result

'2.2 RAG Workflow\n2.2 RAG Workflow\nThe RAG process follows a systematic workflow. When a user submits a query, the system first converts it into an embedding vector. This query embedding is then\nused to search the vector database for the most semantically similar document chunks. The retrieved documents are ranked by relevance, and the top-k results are\nselected as context.\nThe language model receives both the original query and the retrieved context as input, generating a response that synthesizes information from the provided\ndocuments. This approach significantly reduces hallucinations and allows the model to cite sources, improving trustworthiness and verifiability.\n2.3 Advanced RAG Techniques\n2.3 Advanced RAG Techniques\nModern RAG implementations employ sophisticated techniques to improve performance. Hybrid search combines dense vector search with traditional keyword-'

## LangChain Agent

In [76]:
# Define the RAG tool as a LangChain tool for the agent to use

from langchain_core.tools import tool

@tool
def rag_tool(query: str) -> dict:
    """
    Retrieve relevant information from the PDF documents.
    Use this tool when the user asks factual or conceptual questions
    that might be answered from the stored documents.
    
    Args:
        query (str): The user's question or search query
    
    Returns:
        dict: Dictionary containing:
            - query: The original query
            - context: List of relevant document excerpts
            - metadata: Metadata (e.g., page numbers) for each document
    """
    # Retrieve top 4 most similar documents for the query
    result = vector_store.similarity_search(query, k=4)
    context = [doc.page_content for doc in result]
    metadata = [doc.metadata for doc in result]
    
    return {
        "query": query,
        "context": context,
        "metadata": metadata,
    }

In [77]:
tools = [rag_tool]

In [78]:
# - create_agent: Creates a ReAct agent that can use tools
# - MemorySaver: Stores agent state for conversation continuity and checkpointing
from langchain.agents import create_agent
from langgraph.checkpoint.memory import MemorySaver

In [79]:
# Initialize memory saver for storing conversation state and agent history
# This enables multi-turn conversations and state checkpointing
memory = MemorySaver()

In [80]:
# Create a ReAct agent that can use the RAG tool to answer questions
# The agent will automatically decide when to use the tool based on the user's query
agent = create_agent(
    model=llm,                    
    tools=tools,                  
    checkpointer=memory,          
    system_prompt="You are a concise assistant. Always base answers on the provided PDF and clearly cite any document excerpts used."
)

In [81]:
# Set up configuration for agent execution with a specific thread ID
config = {"configurable": {"thread_id": "1"}}

In [82]:
result = agent.invoke(
    {"messages": "Key Components of RAG?"},
    config=config,
)

In [83]:
# Extract the agent's final response (last message) and display it as formatted Markdown
# This provides a clean, readable output of the agent's answer with proper formatting
r = result['messages'][-1].content
from IPython.display import Markdown, display
display(Markdown(r))

**Key components of a Retrieval‑Augmented Generation (RAG) system**

1. **Document Corpus** – the collection of documents, databases, or knowledge bases that contain the information to be retrieved.  
2. **Embedding Model** – converts text (both queries and documents) into dense vector representations that capture semantic meaning.  
3. **Vector Database** – stores the document embeddings and enables fast similarity search (e.g., Pinecone, Weaviate, Chroma).  
4. **Retrieval System** – searches the vector store to find the most relevant document chunks for a given query.  
5. **Language Model** – receives the original query plus the retrieved context and generates the final response.

These components work together to retrieve relevant knowledge and feed it into a generative model, reducing hallucinations and improving answer verifiability【0†L1-L8】.